In [1]:
!pip install streamlit

     |████████████████████████████████| 7.8MB 5.1MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 4.2MB 38.8MB/s 
     |████████████████████████████████| 174kB 42.1MB/s 
     |████████████████████████████████| 112kB 43.9MB/s 
     |████████████████████████████████| 122kB 41.0MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13476 sha256=10225e8af0be53b162477d48e641d47cc57bfa040a1a42baad160b859e1dd39e
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


In [2]:
!pip install pyngrok

     |████████████████████████████████| 747kB 4.9MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19262 sha256=4d0536e1f67946a02c3dfa898185cde19749d06d941c8f7ffd99183635bba891
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok


In [3]:
!touch app.py
!streamlit run app.py &>/dev/null&

In [4]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(8501)
public_url

<NgrokTunnel: "http://83230c179749.ngrok.io" -> "http://localhost:8501">

# [streamlit-bokeh-events](https://github.com/ash2shukla/streamlit-bokeh-events/blob/master/example/custom_js.py)

In [5]:
!pip install streamlit-bokeh-events

     |████████████████████████████████| 2.0MB 6.7MB/s 


# [Code recipes](https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html)

In [6]:
!pip install pyowm

     |████████████████████████████████| 3.1MB 6.7MB/s 


In [7]:
!pip install pydeck

In [10]:
%%writefile app.py
import json
import streamlit as st
import pandas as pd
import pydeck as pdk

from bokeh.models.widgets import Button
from bokeh.models import CustomJS
from streamlit_bokeh_events import streamlit_bokeh_events
from pyowm.owm import OWM

from decimal import Decimal, Context, ROUND_HALF_UP, setcontext
#import os
#os.environ['MAPBOX_API_KEY'] = "pk.eyJ1Ijoia2Fub3VrIiwiYSI6ImNrcTdzNzczZzA4ejQydm8yZWZva2ZpZnMifQ.FqVTDGMSvI4gyRl0ngNmZg"
#os.environ['MAPBOX_ACCESS_TOKEN'] = "pk.eyJ1Ijoia2Fub3VrIiwiYSI6ImNrcTdzNzczZzA4ejQydm8yZWZva2ZpZnMifQ.FqVTDGMSvI4gyRl0ngNmZg"

# Contexを作成
my_context = Context(prec = 64, rounding = ROUND_HALF_UP)

# 作成したContext環境への切り替え
setcontext(my_context)

# https://stackoverflow.com/questions/7490660/converting-wind-direction-in-angles-to-text-words
# https://humet.sci.hokudai.ac.jp/~meteo/tutorial.html
# http://snowfence.umn.edu/Components/winddirectionanddegrees.htm
def degToCompass(num):
    val=int((num/22.5)+.5)
    #   "北","北北東","北東","東北東","東","東南東","南東","南南東", "南","南南西","南西","西南西","西","西北西","北西","北北西"
    arr=["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]
    # print arr[(val % 16)]
    return arr[(val % 16)]

#owm = OWM('3671853e8d7d534a172ca6f753869e26') 
#reg = owm.city_id_registry()
#list_of_locations = reg.locations_for('tokyo', country='JP')
#tokyo = list_of_locations[0]
##lat = tokyo.lat
##lon = tokyo.lon

@st.cache
def load_data(lat, lon):
    #owm = OWM('your-free-api-key')
    owm = OWM('3671853e8d7d534a172ca6f753869e26')    
    mgr = owm.weather_manager()

    #lat = tokyo.lat
    #lon = tokyo.lon

    # https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#onecall-data
    # https://pyowm.readthedocs.io/en/latest/_modules/pyowm/weatherapi25/weather.html
    # https://openweathermap.org/api/one-call-api
    one_call = mgr.one_call(lat=lat, lon=lon)
    current = one_call.current

    geo_mgr = owm.geocoding_manager()
    locations = geo_mgr.reverse_geocode(lat=lat, lon=lon, limit=1)
    city = locations[0].name
    country = locations[0].country

    #st.write(current)

    # "current":{"dt":1622199425,"sunrise":1622143696,"sunset":1622195305,"temp":21.43,"feels_like":21.3,"pressure":1002,"humidity":64,"dew_point":14.34,"uvi":0,"clouds":75,"visibility":10000,"wind_speed":1.34,"wind_deg":254,"wind_gust":2.68,"weather":[{"id":803,"main":"Clouds","description":"曇りがち","icon":"04n"}]},
    temp = int(Decimal(str(current.temperature('celsius').get('temp'))).quantize(Decimal("0.1")))
    press = current.pressure.get('press')
    humidity = current.humidity
    clouds = current.clouds
    speed = float(Decimal(str(current.wind()['speed'])).quantize(Decimal("0.1")))
    deg = current.wind()['deg']
    compass = degToCompass(deg)
    status = current.status
    uvi = int(Decimal(str(current.uvi)).quantize(Decimal("0")))

    # https://www.codesansar.com/python-programming-examples/convert-temperature-celsius-fahrenheit-kelvin.htm
    # https://docs.python.org/ja/3/library/decimal.html
    dewpoint = int(Decimal(str(current.dewpoint - 273.15)).quantize(Decimal("0")))
    visibility_distance = format(float(Decimal(str(current.visibility_distance / 1000)).quantize(Decimal("0.1"))), '.1f')
    detailed_status = current.detailed_status
    weather_icon_name = current.weather_icon_name
    weather_icon_url = "http://openweathermap.org/img/wn/" + weather_icon_name + "@2x.png"

    df = pd.DataFrame(
        [[country, city, lat, lon, temp, press, humidity, clouds, speed, compass, uvi, dewpoint, visibility_distance, status, detailed_status, weather_icon_name, weather_icon_url]],
        columns=['country', 'city', 'lat', 'lon', 'temp', 'pressure', 'humidity', 'clouds', 'speed', 'compass', 'uvi', 'dewpoint', 'visibility_distance', 'status', 'detailed_status', 'weather_icon_name', 'weather_icon_url'])
    return df

loc_button = Button(label="Get Location")
loc_button.js_on_event("button_click", CustomJS(code="""
    navigator.geolocation.getCurrentPosition(
        (loc) => {
            document.dispatchEvent(new CustomEvent("GET_LOCATION", {detail: {lat: loc.coords.latitude, lon: loc.coords.longitude}}))
        }
    )
    """))
result = streamlit_bokeh_events(
    loc_button,
    events="GET_LOCATION",
    key="get_location",
    refresh_on_update=False,
    override_height=75,
    debounce_time=0)

if result:
    if "GET_LOCATION" in result:
        st.write(result.get("GET_LOCATION"))

        lat = result.get("GET_LOCATION")["lat"]
        lon = result.get("GET_LOCATION")["lon"]

        df = load_data(lat, lon)
        
        st.pydeck_chart(pdk.Deck(
            map_style='mapbox://styles/mapbox/streets-v11',
            #map_style='mapbox://styles/kanouk/ckq7sdv9b638417pjuslhnwqr',
            #api_keys={"mapbox": "pk.eyJ1Ijoia2Fub3VrIiwiYSI6ImNrcTdzNzczZzA4ejQydm8yZWZva2ZpZnMifQ.FqVTDGMSvI4gyRl0ngNmZg"},
            initial_view_state=pdk.ViewState(
                latitude=lat,
                longitude=lon,
                zoom=10.5,
                pitch=50,
            ),
            layers=[
                pdk.Layer(
                    'ScatterplotLayer',
                    data=df,
                    pickable=True,
                    get_position='[lon, lat]',
                    get_fill_color='[255, 32, 32, 160]',
                    get_radius=1000,
                ),
            ],
            tooltip={"html": """
            <table>
            <thead><tr><th colspan=\"2\">{city}</th></tr></thead>
            <tbody>
            <tr><td><img src=\"{weather_icon_url}\"></td><td>{status}<br>{detailed_status}</td></tr>
            <tr><td>country</td><td>{country}</td></tr>
            <tr><td>temp</td><td>{temp}°C</td></tr>
            <tr><td>clouds</td><td>{clouds}%</td></tr>
            <tr><td>humidity</td><td>{humidity}%</td></tr>
            <tr><td>pressure</td><td>{pressure}hPa</td></tr>
            <tr><td>wind direction</td><td>{compass}</td></tr>
            <tr><td>wind speed</td><td>{speed}m/s</td></tr>
            <tr><td>UV</td><td>{uvi}</td></tr>
            <tr><td>Dew point</td><td>{dewpoint}°C</td></tr>
            <tr><td>Visibility</td><td>{visibility_distance}km</td></tr>
            </tbody>
            </table>
            """, "style": {"background-color": "rgba(160, 211, 255, 0.9)", "color": "white"}},
        ))


Overwriting app.py
